# Best Place for Indian to Stay in Toronto

#### First Import the necessary libraries

In [188]:
!pip install BeautifulSoup4
!pip install folium

In [189]:
import requests
import pandas as pd
import numpy as np
import folium
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from geopy.geocoders import Nominatim
import folium
import requests
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
print("Import Completed")

Import Completed


#### Get the website HTML codes using BeautifulSoup

In [190]:
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,"lxml")

In [191]:
My_table = soup.find("table",{"class":"wikitable sortable"})
print("Table is stored successfully")

Table is stored successfully


#### Convert the table to Pandas Dataframe

In [192]:
A=[]
B=[]
C=[]


for row in My_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].text.rstrip('\n'))
        B.append(cells[1].text.rstrip('\n'))
        C.append(cells[2].text.rstrip('\n'))

In [193]:
import pandas as pd
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Filtering the Datas

Removing the rows with the "Not Assined" values

In [194]:
df = df[df.Borough != "Not assigned"].reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Joining the datas which has the same postal Codes

In [195]:
df = df.groupby(["Postal Code", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [196]:
df.shape[0]

103

#### Adding Lattitude and Longitude to the data Frame

In [197]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
print("Read successfully")

Read successfully


In [198]:
df_new = df.merge(coordinates, on="Postal Code", how="left")
df_new.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Making maps of Toronto Area

In [199]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [200]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

#### Getting Indian Communities nearby based on their restaurants

In [201]:
CLIENT_ID = 'QAASBFHYDZJEBCRPW4VHU3MDEXBDCJGJWSCVDLI0GYZ2SSVK' # your Foursquare ID
CLIENT_SECRET = 'M2DO2PL34K1WHVNOJXLBK4EB2BQX5L3Q2FLLUGI3GRPT4QZK'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QAASBFHYDZJEBCRPW4VHU3MDEXBDCJGJWSCVDLI0GYZ2SSVK
CLIENT_SECRET:M2DO2PL34K1WHVNOJXLBK4EB2BQX5L3Q2FLLUGI3GRPT4QZK


In [202]:
df.loc[df['Postal Code'] == "M5S"]

,Postal Code,Borough,Neighborhood
66,M5S,Downtown Toronto,"University of Toronto, Harbord"


In [203]:
neighborhood_latitude = df_new.loc[66, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_new.loc[66, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_new.loc[66, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of University of Toronto, Harbord are 43.6626956, -79.4000493.


In [204]:
LIMIT = 100
radius = 5000
indian_restaurant_id = "4bf58dd8d48988d10f941735"
url = 'https://api.foursquare.com/v2/venues/explore?categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    indian_restaurant_id,
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?categoryId=4bf58dd8d48988d10f941735&client_id=QAASBFHYDZJEBCRPW4VHU3MDEXBDCJGJWSCVDLI0GYZ2SSVK&client_secret=M2DO2PL34K1WHVNOJXLBK4EB2BQX5L3Q2FLLUGI3GRPT4QZK&v=20180605&ll=43.6626956,-79.4000493&radius=5000&limit=100'

In [205]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5efde257042e2d70f1179c66'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'query': 'indian',
  'totalResults': 87,
  'suggestedBounds': {'ne': {'lat': 43.707695645000044,
    'lng': -79.3379605790831},
   'sw': {'lat': 43.617695554999955, 'lng': -79.46213802091691}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4adb969ef964a520332921e3',
       'name': 'Banjara Indian Cuisine',
       'location': {'address': '796 Bloor St W',
        'crossStreet': 'at Crawford St',
        'lat': 43.66291638533112,
        'lng': -79.4219109810459,
        'labeledLatLngs

In [206]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [207]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.postalCode' ]
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng,postalCode
0,Banjara Indian Cuisine,Indian Restaurant,43.662916,-79.421911,M6G 1L8
1,Kothur Indian Cuisine,Indian Restaurant,43.667872,-79.385659,M4Y
2,The Host,Indian Restaurant,43.669631,-79.395599,M5R 1A9
3,Pukka Restaurant,Indian Restaurant,43.681055,-79.429187,M6C 1B6
4,Roti Cuisine of India,Indian Restaurant,43.674618,-79.408249,M5R 2X8
5,Utsav Indian Cuisine,Indian Restaurant,43.671154,-79.390802,M5R 1B8
6,Gandhi Cuisine,Indian Restaurant,43.647405,-79.403074,M5V 2B5
7,Madras Masala,Indian Restaurant,43.662959,-79.421646,M6G 1L7
8,Little India Restaurant,Indian Restaurant,43.650319,-79.388998,M5V 1Z4
9,King's Palace,Indian Restaurant,43.672214,-79.385843,M4W 2M9


In [208]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, name, categories in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

### Cluster the Indian Places

In [209]:
df_indian_places = nearby_venues

In [210]:
nearby_venues['postalCode'] = nearby_venues['postalCode'].str[:3]
nearby_venues.head()

,name,categories,lat,lng,postalCode
0,Banjara Indian Cuisine,Indian Restaurant,43.662916,-79.421911,M6G
1,Kothur Indian Cuisine,Indian Restaurant,43.667872,-79.385659,M4Y
2,The Host,Indian Restaurant,43.669631,-79.395599,M5R
3,Pukka Restaurant,Indian Restaurant,43.681055,-79.429187,M6C
4,Roti Cuisine of India,Indian Restaurant,43.674618,-79.408249,M5R


In [211]:
to_onehot = pd.get_dummies(df_indian_places[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['postalCode'] = df_indian_places['postalCode'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(87, 18)


,postalCode,Bar,Breakfast Spot,Café,Coffee Shop,Deli / Bodega,Diner,Event Space,Food Court,Grocery Store,Hotel,Indian Restaurant,Italian Restaurant,Market,Middle Eastern Restaurant,North Indian Restaurant,Sri Lankan Restaurant,Vegetarian / Vegan Restaurant
0,M6G,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,M4Y,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,M5R,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,M6C,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,M5R,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [212]:
to_grouped = to_onehot.groupby(["postalCode"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

(24, 18)


,postalCode,Bar,Breakfast Spot,Café,Coffee Shop,Deli / Bodega,Diner,Event Space,Food Court,Grocery Store,Hotel,Indian Restaurant,Italian Restaurant,Market,Middle Eastern Restaurant,North Indian Restaurant,Sri Lankan Restaurant,Vegetarian / Vegan Restaurant
0,M4K,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
1,M4M,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
2,M4P,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
3,M4S,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
4,M4T,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
5,M4W,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
6,M4Y,0.0,0.000000,0.000000,0.000000,0.000000,0.333333,0.0,0.0,0.00,0.000000,0.333333,0.0,0.0,0.000000,0.333333,0.000000,0.0
7,M5A,0.0,0.000000,0.000000,0.000000,0.333333,0.333333,0.0,0.0,0.00,0.000000,0.333333,0.0,0.0,0.000000,0.000000,0.000000,0.0
8,M5C,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
9,M5G,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0


In [213]:
len(to_grouped[to_grouped["Indian Restaurant"] > 0])

21

In [214]:

to_indian = to_grouped[["postalCode","Indian Restaurant"]]
to_indian.head()

,postalCode,Indian Restaurant
0,M4K,1.0
1,M4M,1.0
2,M4P,1.0
3,M4S,1.0
4,M4T,1.0


In [215]:
from sklearn.cluster import KMeans
toclusters = 3

to_clustering = to_indian.drop(["postalCode"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=1)
kmeans.fit_transform(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([0, 0, 0, 0, 0, 0, 1, 1, 2, 0, 1, 1, 0, 0, 2, 1, 0, 0, 2, 0],
      dtype=int32)

In [216]:

to_merged = to_indian.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_

In [217]:

to_merged

,postalCode,Indian Restaurant,Cluster Labels
0,M4K,1.000000,0
1,M4M,1.000000,0
2,M4P,1.000000,0
3,M4S,1.000000,0
4,M4T,1.000000,0
5,M4W,1.000000,0
6,M4Y,0.333333,1
7,M5A,0.333333,1
8,M5C,0.000000,2
9,M5G,1.000000,0


In [218]:
to_merged = to_merged.join(nearby_venues.set_index("postalCode"), on="postalCode")

print(to_merged.shape)
to_merged.head()

(61, 7)


,postalCode,Indian Restaurant,Cluster Labels,name,categories,lat,lng
0,M4K,1.0,0,Butter Chicken Factory,Indian Restaurant,43.667072,-79.369184
0,M4K,1.0,0,Sher-E-Punjab,Indian Restaurant,43.677308,-79.353066
1,M4M,1.0,0,Siddhartha Indian,Indian Restaurant,43.659117,-79.348948
2,M4P,1.0,0,TYC by Sanjeev Kapoor (The Yellow Chilli),Indian Restaurant,43.707034,-79.396109
3,M4S,1.0,0,Khau Gully,Indian Restaurant,43.700540,-79.396927


In [219]:
map_clusters = folium.Map(location=[latitude,longitude],zoom_start=12)

# set color scheme for the clusters


# add markers to the map
markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'green'
markers_colors[3] = 'yellow'
markers_colors[4] = 'cyan'
markers_colors[5] = 'black'
for lat, lon, cluster in zip(to_merged['lat'], to_merged['lng'], to_merged['Cluster Labels']):
    folium.CircleMarker(
        [lat, lon],
        radius=5,
       
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [220]:
#Cluster 0
to_merged.loc[(to_merged['Cluster Labels'] ==0) & (to_merged['categories'] == 'Indian Restaurant') ]

,postalCode,Indian Restaurant,Cluster Labels,name,categories,lat,lng
0,M4K,1.00,0,Butter Chicken Factory,Indian Restaurant,43.667072,-79.369184
0,M4K,1.00,0,Sher-E-Punjab,Indian Restaurant,43.677308,-79.353066
1,M4M,1.00,0,Siddhartha Indian,Indian Restaurant,43.659117,-79.348948
2,M4P,1.00,0,TYC by Sanjeev Kapoor (The Yellow Chilli),Indian Restaurant,43.707034,-79.396109
3,M4S,1.00,0,Khau Gully,Indian Restaurant,43.700540,-79.396927
4,M4T,1.00,0,Chef of India,Indian Restaurant,43.687391,-79.393663
5,M4W,1.00,0,King's Palace,Indian Restaurant,43.672214,-79.385843
5,M4W,1.00,0,Sukhi Biryani House,Indian Restaurant,43.669478,-79.385976
5,M4W,1.00,0,Esther's Soup Kitchen,Indian Restaurant,43.670991,-79.388014
5,M4W,1.00,0,Randys Roti & Doubles,Indian Restaurant,43.672560,-79.387765


In [221]:
#Cluster 1
to_merged.loc[(to_merged['Cluster Labels'] ==1) & (to_merged['categories'] == 'Indian Restaurant') ]

,postalCode,Indian Restaurant,Cluster Labels,name,categories,lat,lng
6,M4Y,0.333333,1,Kothur Indian Cuisine,Indian Restaurant,43.667872,-79.385659
7,M5A,0.333333,1,Butter Chicken Roti,Indian Restaurant,43.650820,-79.369830
10,M5H,0.500000,1,Maezo Restaurant & Bar,Indian Restaurant,43.651091,-79.381843
11,M5J,0.666667,1,Indian Roti House,Indian Restaurant,43.639060,-79.385422
11,M5J,0.666667,1,Tamarind: The Indian Kitchen,Indian Restaurant,43.646859,-79.378707
15,M5V,0.666667,1,Gandhi Cuisine,Indian Restaurant,43.647405,-79.403074
15,M5V,0.666667,1,Little India Restaurant,Indian Restaurant,43.650319,-79.388998
15,M5V,0.666667,1,My Roti Place,Indian Restaurant,43.648663,-79.397306
15,M5V,0.666667,1,Aroma Fine Indian Restaurant,Indian Restaurant,43.646463,-79.389644
15,M5V,0.666667,1,259 Host,Indian Restaurant,43.644869,-79.390837


In [222]:
#Cluster 2
to_merged.loc[(to_merged['Cluster Labels'] ==2) & (to_merged['categories'] == 'Indian Restaurant') ]

,postalCode,Indian Restaurant,Cluster Labels,name,categories,lat,lng


In [223]:
#Cluster 3
to_merged.loc[(to_merged['Cluster Labels'] ==3) & (to_merged['categories'] == 'Indian Restaurant') ]

,postalCode,Indian Restaurant,Cluster Labels,name,categories,lat,lng


In [224]:
#Cluster 4
to_merged.loc[(to_merged['Cluster Labels'] ==4) & (to_merged['categories'] == 'Indian Restaurant') ]

,postalCode,Indian Restaurant,Cluster Labels,name,categories,lat,lng


### Conclusion



Most of the indian communities are present on cluster 1. Hence it is comfortable for the people who are moving to Toronto can stay in places listed below

- East Toronto
- Central Toronto
- Downtown Toronto